In [2]:
import pandas as pd

In [7]:
df = pd.read_csv('/Users/pratheeshjp/Documents/course-registration-chatbot/data_pipeline/notebooks/data/courses.csv')

In [10]:
print(df.head())

     CRN Campus Description                           Course Title  \
0  34154             Online  Computer Science and Its Applications   
1  32828             Boston                        Lab for CS 1100   
2  32829             Boston                        Lab for CS 1100   
3  32830             Boston                        Lab for CS 1100   
4  34155             Online                     First Year Seminar   

  Subject Course       Faculty Name  \
0         CS1100   Lieberherr, Karl   
1         CS1101   Lieberherr, Karl   
2         CS1101   Lieberherr, Karl   
3         CS1101   Lieberherr, Karl   
4         CS1200  Wassinger, Claire   

                                  Course Description         Term  Begin Time  \
0  Introduces students to the field of computer s...  Spring 2025         NaN   
1  Accompanies CS 1100. Involves experiments and ...  Spring 2025       800.0   
2  Accompanies CS 1100. Involves experiments and ...  Spring 2025       915.0   
3  Accompanies CS 11

In [20]:
df.shape

(500, 11)

In [32]:
import pandas as pd

def clean_time(time_val):
    if pd.isna(time_val) or time_val == '' or time_val == 0:
        return None
    return str(int(time_val)).zfill(4) 


def format_time(time_str):
    if not time_str or len(time_str) != 4:
        return None
    hours = int(time_str[:2])
    minutes = time_str[2:]
    period = "AM" if hours < 12 else "PM"
    if hours > 12:
        hours -= 12
    elif hours == 0:
        hours = 12
    return f"{hours}:{minutes} {period}"



def course_to_sentence(row):
    crn = str(row.get('CRN', ''))
    campus = row.get('Campus Description', '')
    title = row.get('Course Title', '')
    subject_course = row.get('Subject Course', '')
    faculty = row.get('Faculty Name', '')
    description = row.get('Course Description', '')
    term = row.get('Term', '')

    begin_time = clean_time(row.get('Begin Time', ''))
    end_time = clean_time(row.get('End Time', ''))
    days = row.get('Days', '')
    prerequisites = row.get('Prerequisites', '[]')


    sentence_parts = [f"{subject_course} (CRN: {crn}), {title},"]
    

    if campus.lower() == 'online':
        sentence_parts.append("is an online course")
    elif campus.lower() == 'no campus, no room needed':
        sentence_parts.append("is a self-paced course")
    else:
        sentence_parts.append(f"is offered at {campus}")
    
    
    sentence_parts.append(f"for {term}")
    

    if begin_time and end_time and days:
        formatted_begin = format_time(begin_time)
        formatted_end = format_time(end_time)
        schedule = f"with classes scheduled {days} from {formatted_begin} to {formatted_end}"
        sentence_parts.append(schedule)
    

    if faculty:
        sentence_parts.append(f"taught by Professor {faculty}")
    

    sentence = " ".join(sentence_parts) + "."
    

    if description:
        description = ' '.join(str(description).split())
        sentence += f" {description}"
    
  
    if prerequisites == '[]' or not prerequisites or prerequisites.strip() == '':
        sentence += " This course is open to all students with no prerequisites required."
    else:
        prereqs = prerequisites.strip('[]').replace("'", "").replace('"', '')
        if prereqs:
            sentence += f" Prerequisites for this course: {prereqs}."
    
   
    return ' '.join(sentence.split())

def process_course_data(file_path):
    """Process entire course dataset and convert to sentences."""
    df = pd.read_csv(file_path)
    return [course_to_sentence(row) for _, row in df.iterrows()]




In [33]:
course_sentences = process_course_data('/Users/pratheeshjp/Documents/course-registration-chatbot/data_pipeline/notebooks/data/courses.csv')

In [40]:
print(course_sentences[400])

CS6965 (CRN: 38204), Co-op Work Experience Abroad, is a self-paced course for Spring 2025 taught by Professor Celestine, Thea. Provides eligible students with an opportunity for work experience abroad. May be repeated without limit. This course is open to all students with no prerequisites required.
